In [ ]:
# @title ライブラリのインポート

import colorsys
import os

import cv2
import numpy as np
from pydantic import BaseModel
from tqdm.notebook import tqdm




ライブラリのバージョンの確認方法
```
# OpenCVのバージョン
import cv2
print(f"OpenCV (cv2) バージョン: {cv2.__version__}")

# NumPyのバージョン
import numpy as np
print(f"NumPy バージョン: {np.__version__}")

# Matplotlibのバージョン
import matplotlib
print(f"Matplotlib バージョン: {matplotlib.__version__}")

# pydanticのバージョン
import pydantic
print(f"pydantic バージョン: {pydantic.__version__}")

# tqdmのバージョン
import tqdm
print(f"tqdm バージョン: {tqdm.__version__}")
```

```
OpenCV (cv2) バージョン: 4.10.0
NumPy バージョン: 1.26.4
Matplotlib バージョン: 3.10.0
pydantic バージョン: 2.10.4
tqdm バージョン: 4.67.1
```

# パイプラインの準備

In [ ]:
# @title Configオブジェクト（設定値を格納するオブジェクト）


class Config(BaseModel):
    """
    動画処理に関する設定パラメータを管理するクラス
    """

    # 入力動画に関する設定
    video_dir: str
    """動画ファイルが保存されているディレクトリのパス
    例: '/content/videos/' や 'C:/Users/username/videos/'
    """

    video_filename: str
    """処理対象の動画ファイル名
    例: 'input.mp4' や 'sample_video.avi'
    """

    motion_detector_th_size: float
    """動き検出の閾値サイズ
    小さすぎると細かなノイズも検出し、大きすぎると重要な動きを見逃す可能性がある
    """

    output_video_fps: float
    """出力動画のフレームレート（1秒あたりのフレーム数）
    """

    output_video_size: tuple[int, int]
    """出力動画の解像度を(幅, 高さ)のタプルで指定
    """

    @property
    def video_src_path(self) -> str:
        """
        入力動画ファイルの完全パスを取得するプロパティ

        Returns:
            str: 動画ファイルの完全パス
                 例: '/content/videos/input.mp4'
        """
        return os.path.join(self.video_dir, self.video_filename)


In [ ]:
# @title BBoxオブジェクト（画像上の矩形領域（バウンディングボックス）を表現する）


class BBox(BaseModel):
    """
    画像上の矩形領域（バウンディングボックス）を表現するクラス
    BaseModelを継承して、座標値のバリデーションを行う

    座標系:
    - 原点(0,0)は画像の左上
    - x軸は右方向が正
    - y軸は下方向が正
    """

    left: float
    """矩形の左端のx座標（ピクセル単位）
    例: left=100.0 は画像の左端から100ピクセルの位置
    """

    top: float
    """矩形の上端のy座標（ピクセル単位）
    例: top=50.0 は画像の上端から50ピクセルの位置
    """

    right: float
    """矩形の右端のx座標（ピクセル単位）
    必ず left < right となるべき
    """

    bottom: float
    """矩形の下端のy座標（ピクセル単位）
    必ず top < bottom となるべき
    """

    @property
    def ltrb(self) -> tuple[float, float, float, float]:
        """
        バウンディングボックスの4つの座標値をタプルで取得するプロパティ

        Returns:
            tuple[float, float, float, float]:
                (left, top, right, bottom)の順での座標値のタプル
        """
        return self.left, self.top, self.right, self.bottom


In [ ]:
# @title Detectionオブジェクト（結果を格納するオブジェクト）


class Detection:
    """
    検出結果を扱うクラス
    バウンディングボックスの情報を保持し、画像上に描画する機能を提供
    """

    bbox: BBox | None
    """検出された物体の境界ボックス
    検出結果が存在しない場合はNone
    """

    def draw(self, img: np.ndarray):
        """
        検出結果を画像上に描画するメソッド

        Args:
            img: 描画対象の画像（OpenCV形式、BGR）

        Note:
            内部で_draw_bboxを呼び出してバウンディングボックスを描画
        """
        self._draw_bbox(img)

    def _draw_bbox(self, img: np.ndarray):
        """
        バウンディングボックスを画像上に描画する内部メソッド

        Args:
            img: 描画対象の画像（OpenCV形式、BGR）

        Note:
            - 緑色(BGR=(0,255,0))の矩形を描画
            - 線の太さは2ピクセル
            - cv2.rectangleを使用して描画
        """
        cv2.rectangle(
            img,  # 描画対象の画像
            pt1=(self.bbox.left, self.bbox.top),  # 矩形の左上頂点
            pt2=(self.bbox.right, self.bbox.bottom),  # 矩形の右下頂点
            color=(0, 255, 0),  # 緑色（BGR形式）
            thickness=2,  # 線の太さ（ピクセル）
        )


In [ ]:
# @title Driftingオブジェクト（フレームごとの結果を保存するオブジェクト）


class Drifting:
    img: np.ndarray  # 動画のフレーム
    count: int  # 現在のフレームのインデックス番号
    detections: list[Detection]  # 検出結果
    result_img: np.ndarray  # 検出結果を描画したフレーム


In [ ]:
# @title 動画読み込みエレメント


class VideoFileSrc:
    """
    動画ファイルからフレームを読み込むためのクラス
    OpenCVのVideoCapture機能を使用して動画ファイルを扱う
    """

    def __init__(self, config: Config):
        """
        初期化メソッド

        Args:
            config: 動画処理の設定情報を含むConfigオブジェクト
        """
        # フレーム管理用の変数を初期化
        self.frame_num = 0  # 現在のフレーム番号（0からスタート）

        # 動画ファイルのパスを設定
        self.video_src_path = config.video_src_path

        # VideoCaptureオブジェクトを生成して動画ファイルを開く
        self.cap = cv2.VideoCapture(self.video_src_path)

        # 動画の基本情報を取得
        # CAP_PROPプロパティを使用して動画の属性を読み取る
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 総フレーム数
        self.fps = round(self.cap.get(cv2.CAP_PROP_FPS))  # フレームレート
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 高さ（ピクセル）
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 幅（ピクセル）

    def read(self) -> Drifting:
        """
        動画から1フレームを読み込むメソッド

        Returns:
            Drifting: 読み込んだフレーム情報を含むDriftingオブジェクト

        Note:
            - フレームごとに進捗状況をコンソールに表示
            - フレーム番号は自動的にインクリメント
        """
        # 新しいDriftingオブジェクトを生成
        drifting = Drifting()

        # フレームを読み込む
        _, drifting.img = self.cap.read()

        # 現在のフレーム番号を設定
        drifting.count = self.frame_num

        # 次のフレーム用にカウンタをインクリメント
        self.frame_num += 1

        return drifting


In [ ]:
# @title 動画書き出しエレメント


class VideoSink:
    """
    処理済みフレームを動画ファイルとして保存するクラス
    OpenCVのVideoWriterを使用して動画ファイルを生成
    """

    def __init__(self, config: Config):
        """
        初期化メソッド

        Args:
            config: 動画処理の設定情報を含むConfigオブジェクト

        Note:
            MP4形式で出力するようにコーデックを設定
        """
        # 出力ファイルのパスを生成（元のファイル名 + "_out.mp4"）
        self.output_path = self.generate_output_path(config.video_src_path)

        # VideoWriterオブジェクトを初期化
        self.writer = cv2.VideoWriter(
            self.output_path,  # 出力先のファイルパス
            cv2.VideoWriter_fourcc("m", "p", "4", "v"),  # MP4フォーマット用のコーデック
            config.output_video_fps,  # 出力フレームレート（FPS）
            config.output_video_size,  # 出力解像度（幅, 高さ）
        )

    def write(self, drifting: Drifting) -> Drifting:
        """
        1フレームを動画ファイルに書き出すメソッド

        Args:
            drifting: 処理済みフレーム情報を含むDriftingオブジェクト

        Returns:
            Drifting: 入力されたDriftingオブジェクトをそのまま返す

        Note:
            result_imgプロパティに格納された処理済み画像を書き出し
        """
        # 処理済み画像を動画ファイルに書き出し
        self.writer.write(drifting.result_img)
        return drifting

    def release(self):
        """
        VideoWriterのリソースを解放するメソッド

        Note:
            - 動画ファイルの書き出しが完了したら必ず呼び出す
            - リソースの解放を忘れると、ファイルが正しく保存されない可能性がある
        """
        self.writer.release()

    @staticmethod
    def generate_output_path(file_path: str) -> str:
        """
        入力ファイルパスから出力ファイルパスを生成する静的メソッド

        Args:
            file_path: 入力動画ファイルのパス

        Returns:
            str: 出力動画ファイルのパス

        Example:
            入力: "/path/to/video.avi"
            出力: "/path/to/video_out.mp4"
        """
        # 入力ファイルのパスから拡張子を除いた部分を取得
        file_name_without_ext, _ = os.path.splitext(file_path)

        # 新しいファイル名を生成（元のファイル名 + "_out.mp4"）
        new_file_path = f"{file_name_without_ext}_out.mp4"

        return new_file_path


In [ ]:
# @title 動き検知エレメント


class MotionDetector:
    """
    動画内の動きを検出するクラス
    背景差分法とMOG（Mixture of Gaussian）アルゴリズムを使用して
    動いているオブジェクトを検出する
    """

    def __init__(self, config: Config):
        """
        初期化メソッド

        Args:
            config: 動画処理の設定情報を含むConfigオブジェクト

        Note:
            MOGアルゴリズムは、各ピクセルの背景をガウス分布の混合でモデル化し、
            動的な背景に対しても効果的に動体検出を行える
        """
        # MOGアルゴリズムによる背景分離器を生成
        self.model = cv2.bgsegm.createBackgroundSubtractorMOG()

        # 動き検出の閾値サイズを設定（小さすぎるノイズを除去するため）
        self.th_size = config.motion_detector_th_size

    def detect(self, drifting: Drifting) -> Drifting:
        """
        フレーム内の動きを検出するメソッド

        Args:
            drifting: 処理対象フレームを含むDriftingオブジェクト

        Returns:
            Drifting: 動き検出結果を追加したDriftingオブジェクト

        処理の流れ:
            1. 背景差分による動き検出
            2. 輪郭検出
            3. サイズによるフィルタリング
            4. バウンディングボックスの生成
        """
        # 背景差分法で動きのあるピクセルをマスクとして抽出
        mask = self.model.apply(drifting.img)

        # マスクから輪郭を検出
        # RETR_EXTERNAL: 最も外側の輪郭のみを検出
        # CHAIN_APPROX_SIMPLE: 輪郭を直線で近似して記憶効率を向上
        contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

        # 閾値サイズより大きい輪郭のみをフィルタリング
        contours = list(filter(lambda x: cv2.contourArea(x) > self.th_size, contours))

        # 各輪郭を囲む最小の矩形を計算
        bboxes = list(map(lambda x: cv2.boundingRect(x), contours))

        # 各バウンディングボックスをDetectionオブジェクトに変換
        drifting.detections = [self.postprocess(bbox) for bbox in bboxes]

        return drifting

    @staticmethod
    def postprocess(bbox: tuple[int, int, int, int]) -> Detection:
        """
        OpenCVの矩形形式(x, y, w, h)をDetectionオブジェクトに変換

        Args:
            bbox: OpenCV形式のバウンディングボックス
                 (左上x座標, 左上y座標, 幅, 高さ)

        Returns:
            Detection: 変換後のDetectionオブジェクト
        """
        detection = Detection()
        detection.bbox = BBox(
            left=bbox[0],  # 左端のx座標
            top=bbox[1],  # 上端のy座標
            right=bbox[0] + bbox[2],  # 左端 + 幅 = 右端
            bottom=bbox[1] + bbox[3],  # 上端 + 高さ = 下端
        )
        return detection


In [ ]:
# @title 結果描画エレメント

GOLDEN_RATIO = 0.618033988749895  # 黄金比（描画色の選択に使用）


class DetectionRenderer:
    """
    検出結果を視覚化するクラス
    黄金比を利用して異なる物体に異なる色を割り当て、バウンディングボックスを描画する
    """

    def __init__(self):
        """初期化メソッド"""
        pass

    def get_color(
        self, idx: int, s: float = 0.8, vmin: float = 0.7
    ) -> tuple[int, int, int]:
        """
        検出物体ごとに異なる色を生成するメソッド
        黄金比を使用して、見分けやすい色の組み合わせを生成

        Args:
            idx: 物体のインデックス
            s: 彩度（0.0-1.0）
            vmin: 最小明度（0.0-1.0）

        Returns:
            tuple[int, int, int]: BGR形式の色情報（各要素は0-255）

        Note:
            黄金比を使用することで、連続する数値でも視覚的に異なる色を生成できる
        """
        # 黄金比を使って色相を計算（0.0-1.0の範囲）
        h = np.fmod(idx * GOLDEN_RATIO, 1.0)

        # 明度を計算（vmin-1.0の範囲）
        v = 1.0 - np.fmod(idx * GOLDEN_RATIO, 1.0 - vmin)

        # HSV色空間からRGB色空間に変換
        r, g, b = colorsys.hsv_to_rgb(h, s, v)

        # RGB値を0-255の整数値に変換してBGR形式で返す
        return int(255 * b), int(255 * g), int(255 * r)

    def add_bbox(
        self,
        result_img: np.ndarray,
        bbox: BBox,
        color: tuple[int] = (255, 0, 0),  # デフォルトは青色
    ):
        """
        バウンディングボックスを画像に描画するメソッド

        Args:
            result_img: 描画対象の画像
            bbox: 描画するバウンディングボックス
            color: 描画色（BGR形式、デフォルトは青）

        Returns:
            np.ndarray: バウンディングボックスが描画された画像
        """
        # バウンディングボックスの座標を整数に変換
        bbox_ = [int(p) for p in bbox.ltrb]

        # OpenCVで矩形を描画
        result_img = cv2.rectangle(
            result_img,
            pt1=(bbox_[2], bbox_[3]),  # 右下点
            pt2=(bbox_[0], bbox_[1]),  # 左上点
            color=color,  # 描画色
            thickness=2,  # 線の太さ
        )
        return result_img

    def draw_result(self, drifting: Drifting):
        """
        検出結果全体を画像に描画するメソッド

        Args:
            drifting: 検出結果と画像を含むDriftingオブジェクト

        Returns:
            Drifting: 描画結果を追加したDriftingオブジェクト

        Note:
            各検出物体に異なる色を割り当てて描画
        """
        # 入力画像が存在する場合のみ処理
        if drifting.img is not None:
            # 入力画像をコピーして描画用の画像を作成
            result_img: np.ndarray = drifting.img.copy()

            # 各検出結果に対して処理
            for id, detection in enumerate(drifting.detections):
                # 型チェック: 正しいクラスのインスタンスか確認
                assert isinstance(detection, Detection)

                # 物体ごとに異なる色を生成
                color = self.get_color(id)

                # バウンディングボックスが存在する場合は描画
                if detection.bbox is not None:
                    result_img = self.add_bbox(result_img, detection.bbox, color)

            # 描画結果をdriftingオブジェクトに保存
            drifting.result_img = result_img

        return drifting


# メイン処理

下記のURLから動画ファイルをダウンロードしてください。サイズは1920×1080にしてください。ダウンロード後、Google Colabにアップロードしてください。



**[動画のURL](https://pixabay.com/ja/videos/%E8%B5%B0%E3%82%8B-%E3%83%95%E3%82%A3%E3%83%BC%E3%83%AB%E3%83%89-%E7%94%B7-%E4%BA%BA-45711/)**



> <a href="https://pixabay.com/ja//?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=45711">Pixabay</a>が提供する<a href="https://pixabay.com/ja/users/sergo75-75-14395311/?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=45711">Sergey Semenov</a>の動画


In [ ]:
# @title 動画ファイルのパスの指定

video_dir = "/content"  # @param {type:"string"}
video_filename = "45711-446485467_small.mp4"  # @param {type:"string"}


In [ ]:
# @title 動き検出器のパラメータ設定

motion_detector_th_size = 100.0  # @param


In [ ]:
# @title 出力動画の設定

output_video_fps = 30.0  # @param
output_video_size = (1920, 1080)  # @param


In [ ]:
# @title 設定値の統合

config = Config(
    video_dir=video_dir,  # 動画ファイルのディレクトリ
    video_filename=video_filename,  # 動画ファイル名
    motion_detector_th_size=motion_detector_th_size,  # 動き検出の閾値
    output_video_fps=output_video_fps,  # 出力フレームレート
    output_video_size=output_video_size,  # 出力解像度
)


In [ ]:
# @title 処理エレメントの初期化

video_src = VideoFileSrc(config)  # 動画入力
video_sink = VideoSink(config)  # 動画出力
motion_detector = MotionDetector(config)  # 動き検出
detection_renderer = DetectionRenderer()  # 結果描画


In [ ]:
# @title メインの処理ループ

# フレームごとの処理を開始
for _ in tqdm(range(video_src.frame_count)):
    # 1. フレームの読み込み
    drifting = video_src.read()
    if drifting.img is None:  # 動画終了のチェック
        break

    # 2. 動き検出の実行
    drifting = motion_detector.detect(drifting)

    # 3. 検出結果の描画
    drifting = detection_renderer.draw_result(drifting)

    # 4. 結果の書き出し
    drifting = video_sink.write(drifting)

# 5. 終了処理
video_sink.release()


  0%|          | 0/256 [00:00<?, ?it/s]